# 期末專案
## ＊主題：UBIKE公館站可用車輛預測
## ＊組員：羅正翰R07543051
## ＊成功預測未來一小時之可用車輛

# 一、專案簡介：
在台大校園中，大多的師生都是藉由腳踏車來代步。因此，光是在台大校園周圍，就設有7個UBIKE站。每站皆提供許多的腳踏車供大家使用。但許多時候仍供不應求，想騎腳踏車時卻撲了個空，要是能事先知道甚麼時後有車子可以騎就好了。因此，我希望能夠設計出一個神經網路能為UBIKE站點數量作短期預測，讓我們生活上的交通，變得更加便利。

# 二、實作方法：
## 1.數據來源
### (1) APP  Bus+ =>提供台大周圍各站點即時的單車數
### (2) google =>提供台大即時天氣資訊
## 2.數據蒐集
### (1) 撰寫網路爬蟲，每一分鐘上網爬取一筆資訊。
### (2) 利用2個星期時間，爬取約16000筆資料。
## 3.神經網路
### (1) 輸入 => 星期、時、分、秒、降雨機率
### (2) 輸出 => 預測之單車數量
### (3) 模型 => 利用RNN循環式神經網路為主架構進行學習

# 三、模型架構說明
## 1. 訓練多個二分法模型。
![結構說明1](結構說明1.jpg) 
## 2. 串聯多項模型，作為主架構。
![結構說明2](結構說明2.jpg)
## 3. 將預測結果分為：0 , 1 , 2~3 , 4~5 , 6~7 , 8~9 , 10 台以上

# 四、程式碼：
## 1.爬蟲程式
### (1)導入所需套件

In [1]:
from selenium.webdriver.common.keys import Keys 
from selenium import webdriver 
import requests
from bs4 import BeautifulSoup 
import csv,time
import pandas as pd
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt

### (2)讀取現在時間，並將星期更改為數字1~7

In [2]:

#更改時間資訊，將星期轉為數字 : 1~7
def time1():
    
    time_title=np.array(['日期','星期','時','分','秒','時間'])
    date=time.strftime('%Y%m%d')
    a=time.strftime('%a')
    if a=='Mon':
        a=1
        
    elif a=='Tue':
        a=2
        
    elif a=='Wed':
        a=3
    
    elif a=='Thu':
        a=4
        
    elif a=='Fri':
        a=5
    
    elif a=='Sat':
        a=6
    
    elif a=='Sun':
        a=7
    hour=time.strftime('%H')
    minute=time.strftime('%M')
    second=time.strftime('%S')
    time1=time.strftime('%H%M%S')
    
    data=np.array([date,a,hour,minute,second,time1])
    data=data.reshape(1,6)
    
    data=pd.DataFrame(data,columns=time_title)
    
    return data


### (3)爬取台大即時天氣資訊

In [3]:

#抓取天氣資訊
def weather(html):
    driver = webdriver.Chrome()
    first_url = html 
    driver.get(first_url) 
    time.sleep(2) #等待兩秒卻保有載入資料

    soup = BeautifulSoup(driver.page_source,'html.parser')
    rain=soup.find('span',{'id':'wob_pp'})
    
    data=str(rain.get_text())
    data=data.rstrip('%')
    data=int(data)*0.01
        
    probability=np.zeros([1,1])
    probability[0]=data
    
    driver.close()
    
    weather_title=np.array(['下雨機率'])
    
    data=pd.DataFrame(probability,columns=weather_title)
    
    return data


### (4)爬取台大周圍UBIKE站即時車輛資訊

In [4]:
#抓取台大周圍UBIKE各站之即時車輛資訊

def ubike(html):

    driver = webdriver.Chrome()
    first_url = html 
    driver.get(first_url) 
    time.sleep(2) #等待兩秒卻保有載入資料
    
    soup = BeautifulSoup(driver.page_source,'html.parser')
    area=soup.find_all('tr',{'data-area':'大安區'})
    
    #==============================================
    #羅斯福新生南路站
    
    station1=area[25].find_all('td',{'style':''})

    #可租借的腳踏車(available_bike)
    a_b1=int(station1[2].get_text())

    #可停放的位置(available_stop)
    a_s1=int(station1[3].get_text())

    #===============================================
    #公館站
    station2=area[10].find_all('td',{'style':''})

    #可租借的腳踏車(available_bike)
    a_b2=int(station2[2].get_text())

    #可停放的位置(available_stop)
    a_s2=int(station2[3].get_text())

    #===============================================
    #台灣科技大學站
    station3=area[15].find_all('td',{'style':''})

    #可租借的腳踏車(available_bike)
    a_b3=int(station3[2].get_text())

    #可停放的位置(available_stop)
    a_s3=int(station3[3].get_text())

    #===============================================
    #基隆長興路口站
    station4=area[4].find_all('td',{'style':''})

    #可租借的腳踏車(available_bike)
    a_b4=int(station4[2].get_text())

    #可停放的位置(available_stop)
    a_s4=int(station4[3].get_text())

    #===============================================
    #大安運動中心站
    station5=area[33].find_all('td',{'style':''})

    #可租借的腳踏車(available_bike)
    a_b5=int(station5[2].get_text())

    #可停放的位置(available_stop)
    a_s5=int(station5[3].get_text())

    #===============================================
    #台大資訊大樓站
    station6=area[7].find_all('td',{'style':''})

    #可租借的腳踏車(available_bike)
    a_b6=int(station6[2].get_text())

    #可停放的位置(available_stop)
    a_s6=int(station6[3].get_text())

    #===============================================
    #辛亥新生路口站
    station7=area[5].find_all('td',{'style':''})

    #可租借的腳踏車(available_bike)
    a_b7=int(station7[2].get_text())

    #可停放的位置(available_stop)
    a_s7=int(station7[3].get_text())
    
    a_b_list=list([a_b1,a_b2,a_b3,a_b4,a_b5,a_b6,a_b7])
    
    a_s_list=list([a_s1,a_s2,a_s3,a_s4,a_s5,a_s6,a_s7])
    
    data=np.array(a_b_list+a_s_list)
    data=data.reshape(2,7)
    
    driver.close()
    
    return data

### (4)網路爬蟲主程式

In [ ]:
title=np.array(['日期','星期','時','分','秒','時間','下雨機率',
                '羅斯福新生南路','公館','台灣科技大學','基隆長興路口',
                '大安運動中心','台大資訊大樓','辛亥新生路口'])

a_bf=pd.DataFrame(columns=title)
a_sf=pd.DataFrame(columns=title)

count=0

while True :
    
    time_data=time1()
    
    weather_data=weather('https://www.google.com/search?hl=zh-TW&source=hp&ei=fjHRXJ5cmOTzBbSTisgN&q=%E5%8F%B0%E5%A4%A7+%E5%A4%A9%E6%B0%A3&oq=&gs_l=psy-ab.1.0.35i39l6.0.0..4314...1.0..0.490.490.4-1......0......gws-wiz.....6.nzCpsl5uYOs')
    
    ubike_data=ubike('https://taipei.youbike.com.tw/station/list')
    
    a_b_array=ubike_data[0,:]
    a_b_array=a_b_array.reshape(1,7)
    
    a_s_array=ubike_data[1,:]
    a_s_array=a_s_array.reshape(1,7)
    
    station_title=np.array(['羅斯福新生南路','公館','台灣科技大學',
                  '基隆長興路口','大安運動中心','台大資訊大樓','辛亥新生路口'])
    
    a_bf1=pd.DataFrame(a_b_array,columns=station_title)
    a_sf1=pd.DataFrame(a_s_array,columns=station_title)
    
    a_bf1=pd.concat([time_data,weather_data,a_bf1],axis=1)
    a_sf1=pd.concat([time_data,weather_data,a_sf1],axis=1)
    
    
    a_bf=pd.concat([a_bf,a_bf1],axis=0)
    a_sf=pd.concat([a_sf,a_sf1],axis=0)

    
    count=count+1
    
    clear_output(wait=True)
    
    print('以抓取',count,'筆資料')
    
    a_bf.to_csv('5.14_05_a_bf.csv', index = False)
    a_sf.to_csv('5.14_05_a_sf.csv', index = False)


## 2.子神經網路訓練
### (1)導入所需套件

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.utils import np_utils
%env KERAS_BACKEND=tensorflow

from keras.models import Sequential,Model
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras import optimizers
from keras import backend as K

env: KERAS_BACKEND=tensorflow


### (2)將資料做2分法，大於等於n台車為一類，小於n台車為一類。

In [7]:
def category(data,num):
    
    length=len(data)
    
    category=np.ones(length)
    
    for i in range(0,length):
        if (data[i] == num):
                
            category[i]=0
                
    
    category=category.reshape(length,1)
    category = pd.DataFrame(category, columns=['公館分類'])

    return category

### (3)將資料區分成training data 及 testing data

In [8]:
def sample(x_data,y_data,test_num,random):
    real_test_num=100
    if random==True:
        x_data = x_data.sample(frac=1)
        y_data = y_data.sample(frac=1)
    
    x_data=np.array(x_data)
    y_data=np.array(y_data)
    
    data_num=len(x_data)
    
    x_real_test=x_data[data_num-real_test_num:data_num]
    y_real_test=y_data[data_num-real_test_num:data_num]
    
    x_test=x_data[data_num-real_test_num-test_num:data_num-real_test_num]
    y_test=y_data[data_num-real_test_num-test_num:data_num-real_test_num]
    
    x_train=x_data[0:data_num-real_test_num-test_num]
    y_train=y_data[0:data_num-real_test_num-test_num]
    
    return x_train,x_test,x_real_test,y_train,y_test,y_real_test

### (4)從讀取到模型訓練

In [ ]:
#===============資料讀取及整理=============
a_bf=pd.read_csv('a_bf.csv')

x1=a_bf['星期']
x2=a_bf['時']
x3=a_bf['分']
x4=a_bf['秒']
x5=a_bf['下雨機率']

y1=category(a_bf.公館,0)

x_data=pd.concat([x1,x2,x3,x4,x5],axis=1)
y_data=y1

x_train,x_test,x_real_test,y_train,y_test,y_real_test=sample(x_data,y_data,1000,False)

y_train_c=np_utils.to_categorical(y_train,2)
y_test_c=np_utils.to_categorical(y_test,2)

#===============神經網路架構建立=============

N = 1000 # 文字要壓到 N 維
M = 1000 # LSTM 有 K 個神經元


model = Sequential()
model.add(Embedding(10000, N))

model.add(LSTM(M,return_sequences=False))

Adm=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, 
                epsilon=None, decay=0.0, amsgrad=False)          

model.add(Dense(2, activation='softmax'))

model.compile(loss='mse',optimizer=Adm,metrics=['accuracy'])

#===============神經網路訓練=============

model.fit(x_train, y_train_c)

#===============將模型儲存=============

model.json=model.to_json()
open('0_model.json','w').write(model.json)
model.save_weights('0_model_weights.h5')

## 3.多層神經網路串聯
### (1)導入所需套件

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import model_from_json
import pandas as pd

### (2)讀取各分類方法之模型

In [ ]:
model_0=model_from_json(open('0_model.json').read())
model_1=model_from_json(open('1_model.json').read())
model_2=model_from_json(open('2_model.json').read())
model_4=model_from_json(open('4_model.json').read())
model_6=model_from_json(open('6_model.json').read())
model_8=model_from_json(open('8_model.json').read())
model_10=model_from_json(open('10_model.json').read())

model_0.load_weights('0_model_weights.h5')
model_1.load_weights('1_model_weights.h5')
model_2.load_weights('2_model_weights.h5')
model_4.load_weights('4_model_weights.h5')
model_6.load_weights('6_model_weights.h5')
model_8.load_weights('8_model_weights.h5')
model_10.load_weights('10_model_weights.h5')

### (3)將上層模型分類之資料下傳至下層模型繼續分類

In [ ]:
def descent(test_data1,model_num):
    model=model_num
    
    df=test_data1
    
    x_test=df[['星期','時','分','秒','下雨機率']]
    x_test=np.array(x_test)
    
    test_predict=model.predict_classes(x_test)
    test_predict=test_predict.reshape(len(test_predict),1)
    
    df.分類=test_predict
    
    condition0=(df.分類==0) #小於
    condition1=(df.分類==1) #大於 等於
    
    first=df[condition0]

    second=df[condition1]

    
    return first,second

### (4)讀取testing data，並進行預測

In [ ]:
#===============testing data 讀取===============

test_data=pd.read_csv('test_data(train).csv')

#===============testing data 預測===============

df=pd.DataFrame( columns=['星期','時','分','秒','下雨機率','數量','分類','正確'])

data_b0,data_a0=descent(test_data,model_0)
if len(data_b0)>0:
    data_b0.分類=0
    condition=(data_b0.數量==0)
    data_b0.正確=condition
    print('0_num:',len(data_b0))
    df=data_b0
else:
    print('0_num:',0)
#=====================================model_0
if len(data_a0)>0:
    data_b2,data_a2=descent(data_a0,model_2)

    if len(data_b2)>0:
        data_b2.分類=1
        condition=(data_b2.數量==1)
        data_b2.正確=condition
        print('1_num:',len(data_b2))
        df=pd.concat([df,data_b2],axis=0)
    else:
        print('1_num:',0)
    #=====================================model_2
    if len(data_a2)>0:
        data_b4,data_a4=descent(data_a2,model_4)

        if len(data_b4)>0:
            data_b4.分類=2.3
            condition1=(data_b4.數量==2)
            condition2=(data_b4.數量==3)
            data_b4.正確=condition1+condition2
            print('2.3_num:',len(data_b4))
            df=pd.concat([df,data_b4],axis=0)
        else:
            print('2.3_num:',0)
        #=====================================model_4
        if len(data_a4)>0:
            data_b6,data_a6=descent(data_a4,model_6)

            if len(data_b6)>0:
                data_b6.分類=4.5
                condition1=(data_b6.數量==4)
                condition2=(data_b6.數量==5)
                data_b6.正確=condition1+condition2
                print('4.5_num:',len(data_b6))
                df=pd.concat([df,data_b6],axis=0)
            else:
                print('4.5_num:',0)
            #=====================================model_6
            if len(data_a6)>0:
                data_b8,data_a8=descent(data_a6,model_8)

                if len(data_b8)>0:
                    data_b8.分類=6.7
                    condition1=(data_b8.數量==6)
                    condition2=(data_b8.數量==7)
                    data_b8.正確=condition1+condition2
                    print('6.7_num:',len(data_b8))
                    df=pd.concat([df,data_b8],axis=0)
                else:
                    print('6.7_num:',0)
                #=====================================model_8
                if len(data_a8)>0:
                    data_b10,data_a10=descent(data_a8,model_10)

                    if len(data_b10)>0:
                        data_b10.分類=8.9
                        condition1=(data_b10.數量==8)
                        condition2=(data_b10.數量==9)
                        data_b10.正確=condition1+condition2
                        print('8.9_num:',len(data_b10))
                        df=pd.concat([df,data_b10],axis=0)
                    else:
                        print('8.9_num:',0)
                    if len(data_a10)>0:
                        data_a10.分類=10
                        condition=(data_a10.數量>9)
                        data_a10.正確=condition
                        print('10_num:',len(data_a10))
                        df=pd.concat([df,data_a10],axis=0)
                    else:
                        print('10_num:',0)

                        
#===============將預測結果，以pandas 按照時間順序 輸出===============

df=df.sort_index()


# 五、預測結果分析及討論
## 1. 預測1000筆testing data(約為24Hr)
### (1) 真實數量及預測數量比較
![預測結果1](predict v.s real2.png)
![預測結果2](predict v.s real.png)
*上圖，為真實數量與預測數量的比較，其中預測數量中的10，代表神經網路判斷為10台以上。

*下圖，將真實數量10台以上的情況，都改設為10，更適合與預測情形比較。

### (2) 觀察準確度隨時間的變化
![預測結果3](acc decay with time_muti-layers.png)

*資料預測的準確度，隨時間的增長而成指數快速下降

*當資料數<20時(約為20分鐘以內)，準確度可達 100%

*當資料數<25時(約為25分鐘以內)，準確度可達 80%

*當資料數<50時(約為50分鐘以內)，準確度可達 60%

## 2. 結果討論

若以準確度60%，作為最低可靠標準。則本專案中，可靠預測範圍為未來50分鐘。

若能加長網路爬蟲爬取資料的時間(1個月、甚至1年)，藉以增加training data的數量，也許testing data的可靠預測範圍能變得更大。

# 附錄
## 1. 嘗試過多種訓練模型(以最終模型表現最佳)
### (1)NN
![神經網路模型1](神經網路模型1.jpg)
### (2)RNN with LSTM
![神經網路模型2](神經網路模型2.jpg)
### (3)RNN with GRU
![神經網路模型3](神經網路模型3.jpg)
### (4)NN +RNN
![神經網路模型4](神經網路模型4.jpg)

## 2. 以不同車數為基準做二分法的表現情形
### (1)0~8
![0~8](acc decay with time_model_0-8.png)
### (2)10~18
![10~18](acc decay with time_model_10-18.png)
### (3)20~30
![20~30](acc decay with time_model_20-30.png)